## Import Libraries

In [1]:
import openvino as ov
import ipywidgets as widgets
import yaml
import cv2
import numpy as np
from ultralytics.utils.plotting import colors
from typing import Tuple

import time

In [2]:
import cv2
print(cv2.__version__)


4.10.0


## Load Model

In [3]:
import ipywidgets as widgets
import openvino as ov

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', index=3, options=('CPU', 'GPU.0', 'GPU.1', 'AUTO'), value='AUTO')

In [4]:
model = core.read_model(model="models/best.xml")
compiled_model = core.compile_model(model = model, device_name = device.value)

input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
print("Input Layer shape:", input_layer.shape)
print("Output Layer shape:", output_layer.shape)

Input Layer shape: [1,3,256,256]
Output Layer shape: [1,6,1344]


# Import Labels

In [5]:
with open('models/metadata.yaml') as info:
    info_dict = yaml.load(info, Loader=yaml.Loader)

labels = info_dict['names']
print(labels)

{0: 'fire', 1: 'smoke'}


# Preprocess

## Function

In [6]:
def letterbox(
    img: np.ndarray,
    new_shape: Tuple[int, int] = (640, 640),
    color: Tuple[int, int, int] = (114, 114, 114),
    auto: bool = False,
    scale_fill: bool = False,
    scaleup: bool = False,
    stride: int = 32,
):
    """
    Resize image and padding for detection. Takes image as input,
    resizes image to fit into new shape with saving original aspect ratio and pads it to meet stride-multiple constraints

    Parameters:
      img (np.ndarray): image for preprocessing
      new_shape (Tuple(int, int)): image size after preprocessing in format [height, width]
      color (Tuple(int, int, int)): color for filling padded area
      auto (bool): use dynamic input size, only padding for stride constrins applied
      scale_fill (bool): scale image to fill new_shape
      scaleup (bool): allow scale image if it is lower then desired input size, can affect model accuracy
      stride (int): input padding stride
    Returns:
      img (np.ndarray): image after preprocessing
      ratio (Tuple(float, float)): hight and width scaling ratio
      padding_size (Tuple(int, int)): height and width padding size


    """
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scale_fill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [7]:
def prepare_data(image, input_layer):
    input_w, input_h = input_layer.shape[2], input_layer.shape[3]

    input_image = letterbox(np.array(image))[0]
    input_image = cv2.resize(input_image, (input_w, input_h))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = input_image/255
    input_image = input_image.transpose(2, 0, 1)
    input_image = np.expand_dims(input_image, 0)

    return input_image

### Run Preprocessing and AI Inference

In [8]:
image = cv2.imread("images/test.jpg")
input_image = prepare_data(image, input_layer)
print(np.shape(input_image))

(1, 3, 256, 256)


In [9]:
output = compiled_model([input_image])[output_layer]

In [10]:
print(output)

[[[     13.592      20.519      21.183 ...      179.78      208.33      233.08]
  [     16.219      12.081      7.4724 ...      235.69      224.86      214.69]
  [     27.487      40.447      43.195 ...      217.93      262.06       157.5]
  [     33.985      24.237       14.96 ...      263.96      303.89      245.01]
  [ 3.3104e-05  5.6359e-05  3.4201e-05 ...  4.5266e-05  4.0379e-05   7.616e-05]
  [ 9.1909e-05  8.7254e-05  3.5896e-05 ...  3.7759e-05  3.5953e-05  7.5515e-05]]]


## Postprocessing

### Functions

In [11]:
def evaluate(output, conf_threshold):

    boxes = []
    scores = []
    label_key = []
    label_index = 0

    for class_ in output[0][4:]:
        for index in range(len(class_)):
            confidence = class_[index]

            if confidence > conf_threshold:
                xcen = output[0][0][index]
                ycen = output[0][1][index]
                w = output[0][2][index]
                h =output[0][3][index]

                xmin = int(xcen - (w/2))
                xmax = int(xcen + (w/2))
                ymin = int(ycen - (h/2))
                ymax = int(ycen + (h/2))

                box = (xmin, ymin, xmax, ymax)
                boxes.append(box)
                scores.append(confidence)
                label_key.append(label_index)
        label_index += 1

    boxes = np.array(boxes)
    scores = np.array(scores)

    return boxes, scores, label_key
                

### Run Postprocessing

In [12]:
conf_threshold =.1
boxes, scores, label_key = evaluate(output, conf_threshold)

In [13]:
print(boxes)
print(scores)
print(label_key)


[[106 146 166 208]
 [103 146 169 209]
 [103 146 169 209]
 [102 147 168 209]
 [102 146 168 209]
 [102 146 169 209]
 [103 147 168 209]
 [101 145 168 209]
 [101 145 168 209]
 [102 146 167 209]]
[    0.20399     0.82178     0.86123     0.81731     0.86296      0.7983      0.8645     0.80706     0.85492     0.53587]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Non-Max Suppression 

In [14]:
def non_max_suppression(boxes, scores, conf_threshold):	
    assert boxes.shape[0] == scores.shape[0]
    # bottom-left origin
    ys1 = boxes[:, 0]
    xs1 = boxes[:, 1]
    # top-right target
    ys2 = boxes[:, 2]
    xs2 = boxes[:, 3]
    # box coordinate ranges are inclusive-inclusive
    areas = (ys2 - ys1) * (xs2 - xs1)
    scores_indexes = scores.argsort().tolist()
    boxes_keep_index = []
    while len(scores_indexes):
        index = scores_indexes.pop()
        boxes_keep_index.append(index)
        if not len(scores_indexes):
            break
        ious = compute_iou(boxes[index], boxes[scores_indexes], areas[index],
                           areas[scores_indexes])
        filtered_indexes = set((ious > conf_threshold).nonzero()[0])
        # if there are no more scores_index
        # then we should pop it
        scores_indexes = [
            v for (i, v) in enumerate(scores_indexes)
            if i not in filtered_indexes
        ]
    return np.array(boxes_keep_index)


def compute_iou(box, boxes, box_area, boxes_area):
    # this is the iou of the box against all other boxes
    assert boxes.shape[0] == boxes_area.shape[0]
    # get all the origin-ys
    # push up all the lower origin-xs, while keeping the higher origin-xs
    ys1 = np.maximum(box[0], boxes[:, 0])
    # get all the origin-xs
    # push right all the lower origin-xs, while keeping higher origin-xs
    xs1 = np.maximum(box[1], boxes[:, 1])
    # get all the target-ys
    # pull down all the higher target-ys, while keeping lower origin-ys
    ys2 = np.minimum(box[2], boxes[:, 2])
    # get all the target-xs
    # pull left all the higher target-xs, while keeping lower target-xs
    xs2 = np.minimum(box[3], boxes[:, 3])
    # each intersection area is calculated by the
    # pulled target-x minus the pushed origin-x
    # multiplying
    # pulled target-y minus the pushed origin-y
    # we ignore areas where the intersection side would be negative
    # this is done by using maxing the side length by 0
    intersections = np.maximum(ys2 - ys1, 0) * np.maximum(xs2 - xs1, 0)
    # each union is then the box area
    # added to each other box area minusing their intersection calculated above
    unions = box_area + boxes_area - intersections
    # element wise division
    # if the intersection is 0, then their ratio is 0
    ious = intersections / unions
    return ious

In [15]:
if len(boxes):
    nms_output = non_max_suppression(boxes, scores, conf_threshold)

In [16]:
print(nms_output)

[6]


# Visualize

### Function

In [17]:
def visualize(image, nms_output, boxes, label_key, scores, conf_threshold):
    image_h, image_w, c = image.shape
    input_w, input_h = input_layer.shape[2], input_layer.shape[3]

    for i in nms_output:
        xmin, ymin, xmax, ymax = boxes[i]
        print(xmin, ymin, xmax, ymax)
        
        xmin = int(xmin*image_w/input_w)
        xmax = int(xmax*image_w/input_w)
        ymin = int(ymin*image_h/input_h)
        ymax = int(ymax*image_h/input_h)

        label = label_key[i]
        color = colors(label)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 1)

        font = cv2.FONT_HERSHEY_SIMPLEX
        text = str(int(scores[i]*100)) + "%" +labels[label]
        font_scale = (image_w/1000)
        label_width, label_height = cv2.getTextSize(text, font, font_scale, 1)[0]
        cv2.rectangle(image, (xmin, ymin-label_height), (xmin + label_width, ymin), color, 1)
        
        cv2.putText(image, text, (xmin, ymin), font, font_scale, (255,255,255), 1, cv2.LINE_AA)
    return image

# Combine

In [18]:
def predict_image(image, conf_threshold):
    input_image = prepare_data(image, input_layer)

    start = time.time()
    output = compiled_model([input_image])[output_layer]
    end = time.time()

    inference_time = end - start
    
    
    boxes, scores, label_key = evaluate(output, conf_threshold)

    
    if len(boxes):
        nms_output = non_max_suppression(boxes, scores, conf_threshold)
        visualized_image = visualize(image, nms_output, boxes, label_key, scores, conf_threshold)

        return visualized_image, inference_time
    else:
        return image, inference_time

In [19]:
import cv2
print(cv2.__version__)


4.10.0


In [23]:
image = cv2.imread("images/test.jpg")
conf_threshold = .15
output_image,inference_time = predict_image(image, conf_threshold)

print(inference_time)
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

103 147 168 209
0.007625102996826172


In [28]:
def AddBackground(frame, bg):
    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1]
    ymax = bg.shape[0]
    xmin = xmax - new_w
    ymin = ymax - new_h

    return bg

## Live Demo

In [29]:
def Main():
    camera = cv2.VideoCapture(source)
    inference_average =[]
    bg = cv2.imread(background)
    
    while True:
        ret, frame = camera.read()

        if not ret:
            break

        visualized_frame, inference_time = predict_image(frame, conf_threshold)
        inference_average.append(inference_time)
        deployment = AddBackground(frame, bg)
        
        cv2.imshow('Deployed Model', deployment)
        cv2.imshow('Press Spacebar to Exit', visualized_frame)
        if cv2.waitKey(1) & 0xFF == ord(' '):
            break
    inf_ms = np.average(inference_average)*1000
    print("The average inference time was " + str(np.round(inf_ms,2))+"milliseconds.")
    camera.release()
    cv2.destroyAllWindows()

In [30]:
source = "videos/test.mp4"
background = "background.jpg"
conf_threshold = .4
if __name__ == '__main__':
    Main()

178 139 210 197
40 142 82 195
110 133 208 195
178 139 210 197
40 142 82 195
110 133 208 195
113 74 159 126
37 143 78 175
109 131 210 196
108 78 210 194
174 144 211 198
109 89 171 195
179 144 213 198
110 136 214 197
49 145 78 195
111 74 156 125
108 126 209 196
49 143 81 196
46 145 78 194
109 59 210 195
111 129 208 196
109 134 208 196
176 137 210 197
107 64 165 125
175 155 210 197
117 95 141 124
174 151 211 197
116 92 157 125
176 144 211 197
112 78 164 125
109 98 127 127
109 124 211 197
175 142 210 197
109 137 210 196
110 77 159 123
110 135 210 196
111 80 162 124
43 157 79 196
110 132 208 196
120 61 163 126
112 130 210 196
175 152 210 197
176 145 210 198
110 87 156 126
115 63 160 125
113 127 209 196
34 133 69 173
177 142 212 197
112 131 212 196
110 131 212 197
110 77 149 127
109 75 214 195
47 155 75 195
108 130 217 196
176 148 215 198
45 151 75 195
108 138 219 196
178 140 219 197
44 142 74 195
109 86 130 125
108 129 214 197
45 157 80 196
114 89 163 122
44 151 74 196
111 133 212 196
111 1